In [1]:
import sys, os
sys.path.append('/Users/benjamin/Desktop/repos/chi-data/backend/aws') 
sys.path.append('/Users/benjamin/Desktop/repos/chi-data/backend') 
from s3 import S3
from dynamo import DynamoConn
import pyarrow.parquet as pq
import pandas as pd
import numpy as np
from config import cook_tracts, chicago_tracts, msa_tracts

In [2]:
import s3fs
s3 = s3fs.S3FileSystem()
dataset = 'JT00'
year = '2002'
ds = pq.ParquetDataset(path_or_paths=[
  'bnroths/chicago-data/lehd_od/year=%s/il_lehd_od_main_%s_%s.parquet' % (year, dataset, year), 
  'bnroths/chicago-data/lehd_od/year=%s/il_lehd_od_aux_%s_%s.parquet' % (year, dataset, year), 
    
  'bnroths/chicago-data/lehd_od/year=%s/in_od_main_%s_%s.parquet' % (year, dataset, year), 
  'bnroths/chicago-data/lehd_od/year=%s/in_od_aux_%s_%s.parquet' % (year, dataset, year), 
    
  'bnroths/chicago-data/lehd_od/year=%s/wi_od_main_%s_%s.parquet' % (year, dataset, year), 
  'bnroths/chicago-data/lehd_od/year=%s/wi_od_aux_%s_%s.parquet' % (year, dataset, year), 

], filesystem=s3, validate_schema=False)

In [4]:
table = ds.read()
df = table.to_pandas()


In [6]:
df.head()


,w_geocode,h_geocode,S000,SA01,SA02,SA03,SE01,SE02,SE03,SI01,SI02,SI03,createdate,h_tract,w_tract
0,170010001001001,170010104003020,1,0,1,0,0,0,1,0,0,1,20160228,17001010400,17001000100
1,170010001001003,170010011001025,1,0,1,0,0,0,1,0,1,0,20160228,17001001100,17001000100
2,170010001001003,170010011002036,1,0,1,0,0,0,1,0,1,0,20160228,17001001100,17001000100
3,170010001001003,170010101004121,1,1,0,0,0,1,0,0,1,0,20160228,17001010100,17001000100
4,170010001001003,170010106003018,1,0,0,1,0,0,1,0,1,0,20160228,17001010600,17001000100


In [7]:
df.head()
df = df[df['h_tract'].isin(msa_tracts)]
df = df[df['w_tract'].isin(msa_tracts)]

In [8]:
diff1 = set(df.h_tract) - set(df.w_tract)
for tract in diff1:
	df = df[df.h_tract != tract]

diff2 = set(df.w_tract) - set(df.h_tract)
for tract in diff2:
	df = df[df.w_tract != tract]

In [9]:
diff1 = set(df.h_tract) - set(df.w_tract)
diff2 = set(df.w_tract) - set(df.h_tract)
print diff1, diff2
print df.head()
df.shape

set([]) set([])
              w_geocode        h_geocode  S000  SA01  SA02  SA03  SE01  SE02  \
186220  170310101002000  170318185004010     1     0     1     0     0     0   
186221  170310101002002  170310106001010     1     0     1     0     0     1   
186222  170310101002002  170310106003006     1     0     1     0     0     1   
186223  170310101002002  170310201001020     1     0     1     0     0     1   
186224  170310101002002  170310205002002     1     1     0     0     0     1   

        SE03  SI01  SI02  SI03  createdate      h_tract      w_tract  
186220     1     0     0     1    20160228  17031818500  17031010100  
186221     0     1     0     0    20160228  17031010600  17031010100  
186222     0     1     0     0    20160228  17031010600  17031010100  
186223     0     1     0     0    20160228  17031020100  17031010100  
186224     0     1     0     0    20160228  17031020500  17031010100  


(3706288, 15)

In [10]:
pivot = pd.pivot_table(df, values='SA01', columns=['w_tract'], index=['h_tract'], aggfunc=np.sum, fill_value=0)

In [11]:
print pivot.head()
pivot.shape

# df['S000'].sum()#['17001010400']#.head(n=100) #17001010100

w_tract      17031010100  17031010201  17031010202  17031010300  17031010400  \
h_tract                                                                        
17031010100            4            4            4            4            3   
17031010201            2            0            5            2            3   
17031010202            2            2            2            1            1   
17031010300            3            0            2            9           11   
17031010400            2            1            1            6          102   

w_tract      17031010501  17031010502  17031010503  17031010600  17031010701  \
h_tract                                                                        
17031010100            0            0            0            2            0   
17031010201            1            4            1            0            0   
17031010202            0            4            0            0            0   
17031010300            0            0  

(2207, 2207)

In [81]:

pivot.to_csv('/Users/benjamin/Desktop/repos/chi-data/scripts/test.csv')

In [12]:
w_tracts = pivot.sum()
print w_tracts.head()
h_tracts = pivot.transpose().sum()
A = pivot.transpose()/h_tracts
A = A.fillna(0).replace([np.inf, -np.inf], 0)

w_tract
17031010100      94
17031010201      71
17031010202     120
17031010300     162
17031010400    1866
dtype: int64


In [101]:
print h_tracts.min()
#for a in h_tracts:
#    print a

0


In [13]:
w, v = np.linalg.eig(A)

In [14]:
print type(w[0])
print v[0].shape
print w.shape # eigenvalue
print v.shape # eigenvector

<type 'numpy.complex128'>
(2207,)
(2207,)
(2207, 2207)


In [ ]:
import json
from s3 import S3
s3 = S3()
# print type(w[0])
# print v[0].shape
# print w.shape # eigenvalue
# print v.shape # eigenvector

eigs = []
idx = w.argsort()[::-1]
# print idx
eigenValues = w[idx]
eigenVectors = v[:,idx]
# print eigenValues
# assert False
# for i in range(eigenValues.shape[0]):
sub_data = []
for i in range(20):
    eigenvalue_i = i+1
    val = round(eigenValues[i], 4)
    vector = eigenVectors[:, i]
    sub_data.append({
        "name": "Eigenvalue %s: %s" % (eigenvalue_i, val),
        "key": str(eigenvalue_i)
    })
    # print i, "value", np.linalg.norm(vector), val
    # print "vector"
    # print vector
    # print type(vector.real)
    transformed = [round(1000*x, 1) for x in vector.real]
    eig = {
        "row": i,
        "value": val,
        "vector": transformed,
        "columns": pivot.columns
    }
    eigs.append(eig)
#     print eigs
    tracts = {}
    for i, column in enumerate(pivot.columns):
        tracts[column] = transformed[i]
        
#     print min(tracts.values())
    
    final = {
        "data": tracts,
        "meta": {
            "max": max(tracts.values()),
            "min": min(tracts.values()),
            "top": sorted(tracts.values())[-15]
        }
    }
#     print final
#     break

    stat = eigenvalue_i
    dataset = 'eigs'
    year = '2002'
    with open('%s.json' % stat, 'w') as f:
        f.write(json.dumps(final))

    print '%s.json' % stat
    s3.save_file_public(
        local='%s.json' % stat,
        dataset=dataset, 
        dt=year, 
        filename='%s.json' % stat
    )
    os.remove('%s.json' % stat)

#     break
print json.dumps(sub_data)

In [ ]:

a = np.array([[0, 1], [-2, -3]])
print a
w, v = np.linalg.eig(a)
print v, w

print w[0], v[:,0]
print w[1], v[:,1]

In [ ]:
from s3 import S3
s3 = S3()